In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('CompiledjobListNigeria.csv')

In [11]:
df.head()
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202 entries, 0 to 201
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_title        202 non-null    object
 1   company_name     202 non-null    object
 2   company_desc     202 non-null    object
 3   job_desc         202 non-null    object
 4   job_requirement  202 non-null    object
 5   salary           202 non-null    object
 6   location         202 non-null    object
 7   employment_type  202 non-null    object
 8   department       202 non-null    object
 9   label            202 non-null    int64 
dtypes: int64(1), object(9)
memory usage: 15.9+ KB


job_title          0
company_name       0
company_desc       0
job_desc           0
job_requirement    0
salary             0
location           0
employment_type    0
department         0
label              0
dtype: int64

In [15]:
df_model = df[['employment_type', 'location', 'salary', 'job_title', 'label']]
df_model.head()

,employment_type,location,salary,job_title,label
0,Full Time,Abuja,"75,000 - 150,000",Accountant,0
1,Full Time,Lagos,"60,000 - 100,000",Content Writer,1
2,Full-time,"First Floor, Left Wing, No. 49, Olowu Street, ...",Negotiable,Accountant,1
3,Full Time,Enugu,"75,000 - 150,000",Sales Executive,0
4,Full Time,Lagos,"Less than 75,000",Bulk/Partnership Marketing Officer,0


In [17]:
from sklearn.preprocessing import LabelEncoder

# Create a copy of the dataframe
df_encoded = df_model.copy()

# Initialize LabelEncoder
le = LabelEncoder()

# Encode each categorical column
for col in ['employment_type', 'location', 'salary', 'job_title']:
    df_encoded[col] = le.fit_transform(df_encoded[col])


In [19]:
X = df_encoded.drop('label', axis=1)
y = df_encoded['label']

In [21]:
from sklearn.model_selection import train_test_split

# Split into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Initialize and train model
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8536585365853658
Confusion Matrix:
 [[25  0]
 [ 6 10]]
Classification Report:
               precision    recall  f1-score   support

           0       0.81      1.00      0.89        25
           1       1.00      0.62      0.77        16

    accuracy                           0.85        41
   macro avg       0.90      0.81      0.83        41
weighted avg       0.88      0.85      0.84        41



In [25]:
# Make predictions on the full dataset
df_encoded['ML_Prediction'] = model.predict(X)

# Merge predictions with original dataset (optional for keeping original text)
df_final = pd.concat([df_model, df_encoded['ML_Prediction']], axis=1)

# Preview
df_final.head()

,employment_type,location,salary,job_title,label,ML_Prediction
0,Full Time,Abuja,"75,000 - 150,000",Accountant,0,0
1,Full Time,Lagos,"60,000 - 100,000",Content Writer,1,1
2,Full-time,"First Floor, Left Wing, No. 49, Olowu Street, ...",Negotiable,Accountant,1,1
3,Full Time,Enugu,"75,000 - 150,000",Sales Executive,0,0
4,Full Time,Lagos,"Less than 75,000",Bulk/Partnership Marketing Officer,0,0


In [27]:
# Save to CSV
df_final.to_csv('job_predictions_with_labels.csv', index=False)
print("File saved successfully!")

File saved successfully!


In [35]:
import pandas as pd
# Replace with a known salary value
sample = pd.DataFrame({
    'employment_type': ['Full-time'],
    'location': ['Lagos'],
    'salary': ['200,000 - 300,000'],  # <- from training set
    'job_title': ['Marketer/Sales Executive']
})

# Reuse LabelEncoder from earlier training
from sklearn.preprocessing import LabelEncoder

# Re-initialize encoders (IMPORTANT: same fit order as training!)
le_employment_type = LabelEncoder()
le_location = LabelEncoder()
le_salary = LabelEncoder()
le_job_title = LabelEncoder()

# Fit encoders on the original data
le_employment_type.fit(df_model['employment_type'])
le_location.fit(df_model['location'])
le_salary.fit(df_model['salary'])
le_job_title.fit(df_model['job_title'])

# Apply encoders to the sample
sample_encoded = pd.DataFrame({
    'employment_type': le_employment_type.transform(sample['employment_type']),
    'location': le_location.transform(sample['location']),
    'salary': le_salary.transform(sample['salary']),
    'job_title': le_job_title.transform(sample['job_title'])
})

# Predict using trained model
prediction = model.predict(sample_encoded)

# Output the result
if prediction[0] == 1:
    print("🔴 This job is likely: FAKE")
else:
    print("🟢 This job is likely: REAL")

🔴 This job is likely: FAKE


In [37]:
pip install streamlit

Note: you may need to restart the kernel to use updated packages.


In [39]:
import joblib

# Save the trained model
joblib.dump(model, 'job_model.pkl')

# Save the encoders
joblib.dump(le_employment_type, 'le_employment_type.pkl')
joblib.dump(le_location, 'le_location.pkl')
joblib.dump(le_salary, 'le_salary.pkl')
joblib.dump(le_job_title, 'le_job_title.pkl')

print("Model and encoders saved successfully.")

Model and encoders saved successfully.
